In [1]:
import paho.mqtt.client as mqtt
import time
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
import scipy.stats as stats
import os
from pickle import load

In [2]:
#Helper functiuons

global sample_data


def on_message(client, userdata, message):
    global sample
    global loop
    global msg_count
#     print("message received " ,str(message.payload.decode("utf-8")))
    print("message topic=",message.topic)
    print("message qos=",message.qos)
    print("message retain flag=",message.retain)
    sample = str(message.payload.decode("utf-8"))
    sample_data.extend(get_sample_data())
    msg_count = msg_count + 1
    print('Number of messages recieved: ', msg_count)
    if msg_count == 20:
        loop = False
        client.loop_stop()
    

def MQTT_rec():
    global loop
    while loop==True:
        client.loop_start()
        time.sleep(3)
    client.loop_stop()
    
    
def get_sample_data():
    global sample
    data = []
    test_frames = []
    sample_list = sample.split(';')[:-1]
    for vals in sample_list:
        row = vals.split(',')
        row = list(map(float, row))
        data.append(row)
 
    return data



In [4]:
sample_topic = "MPU6050/Sample"            #MQTT topic that sample data is sent on
broker_address="192.168.1.127"             #broker_address"
client = mqtt.Client("Pool_Pump")          #create new instance
client.on_message=on_message               #attach function to callback
print("connecting to broker")
client.connect(broker_address)             #connect to broker
client.subscribe(sample_topic)             #Subscribe to sample topic

loop=True
sample_data = []                 #Create empty list to hold MQTT sample data
msg_count = 0
frame_size = 100                 #Frame size for data input to model (Must be same shape as train data)
data_folder = 'Inference Data'   #Sample data folder






client.publish("MPU6050/SendSample","1")                           #Send message to esp8266 to sendsample data

MQTT_rec()                                                         #Start MQTT recieve loop and wait for data message


#Check to see if sample data folder is empty and create first file or create next file in sequence
if len(os.listdir(data_folder)) < 1:
    file_name = '1.csv'
    print('No files in class folder... Creating 1.csv')
else:
    name_list = []
    for name in os.listdir(data_folder):
        name =  int(name.split('.')[0])
        name_list.append(name)
    file_name = max(name_list) + 1
    file_name = str(file_name) + '.csv'
        
print('Creating file: ', file_name)                                #Print name of csv file created



### Un-comment the below 3 lines if you want to save data message to file ###

# df = pd.DataFrame(data = sample_data)
# data_file = os.path.join(data_folder,file_name)
# df.to_csv(data_file, index = False, header = False)



columns = ['x', 'y', 'z','Gyx','Gyy','Gyz']                        #CSV column labels
scaler = load(open('scaler.pkl', 'rb'))                            #Load scaled value from file
Test = scaler.transform(sample_data)                               #Scale data
Test_scaled_X = pd.DataFrame(data = Test, columns = columns)       #Convert data to Pandas dataframe


#Generate test frames
test_frames = []
for i in range(0, len(Test_scaled_X) - frame_size, frame_size):
    frame = Test_scaled_X.values[i: i + frame_size]
    test_frames.append(frame)

test_frames = np.asarray(test_frames)                               #Conver to numpy array

#Reshape to a shape that the convolution input layer expects.
X_new = test_frames.reshape(test_frames.shape[0], test_frames.shape[1], test_frames.shape[2], 1)




model = load_model('model_ex.h5')                                   #Load saved model from file
y_pred = np.argmax(model.predict(X_new), axis=-1)                   #Find which class has the max score

print(y_pred)
pred_class = stats.mode(y_pred)[0][0]                               #Determine what is the most common class pred

print('Class:',pred_class)


#The below lines checks the accuracy based on the most common prediction being the correctly predicted class (This does not alway hold true)
unique, counts = np.unique(y_pred, return_counts=True)
outputs = len(y_pred)

for index,i in enumerate(unique):
    if i == pred_class:
        pred_count = counts[index]
        print(pred_count)

acc = float(pred_count/outputs)
print('Prediction accuracy',acc)


#Send MQTT message to broker depending on the class prediction
if pred_class==0:
    client.publish("Pool/Status","Normal - No skimmer")
    print('Normal - No skimmer')
if pred_class==1:
    client.publish("Pool/Status","Normal skimmer operation")  
    print('Normal skimmer operation')
if pred_class==2:
    client.publish("Pool/Status","Skimmer blocked")
    print('Skimmer blocked')
if pred_class==3:
    client.publish("Pool/Status","Filter dirty")
    print('Filter dirty')

connecting to broker
message topic= MPU6050/Sample
message qos= 0
message retain flag= 0
Number of messages recieved:  1
message topic= MPU6050/Sample
message qos= 0
message retain flag= 0
Number of messages recieved:  2
message topic= MPU6050/Sample
message qos= 0
message retain flag= 0
Number of messages recieved:  3
message topic= MPU6050/Sample
message qos= 0
message retain flag= 0
Number of messages recieved:  4
message topic= MPU6050/Sample
message qos= 0
message retain flag= 0
Number of messages recieved:  5
message topic= MPU6050/Sample
message qos= 0
message retain flag= 0
Number of messages recieved:  6
message topic= MPU6050/Sample
message qos= 0
message retain flag= 0
Number of messages recieved:  7
message topic= MPU6050/Sample
message qos= 0
message retain flag= 0
Number of messages recieved:  8
message topic= MPU6050/Sample
message qos= 0
message retain flag= 0
Number of messages recieved:  9
message topic= MPU6050/Sample
message qos= 0
message retain flag= 0
Number of m